# 7. Делаем модель нейросети для распознавания рукописных цифр

In [1]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np

2024-10-15 22:45:59.730527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 22:45:59.741858: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 22:45:59.745053: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [14]:
#x_train = np.asarray(x_train).astype(np.float32)
#y_train = np.asarray(y_train).astype(np.float32)
#x_test = np.asarray(x_test).astype(np.float32)
#y_test = np.asarray(y_test).astype(np.float32)

In [3]:
x_train = x_train / 255
x_test = x_test / 255

In [4]:
x_train = tf.reshape(tf.cast(x_train, tf.float32), [-1, 28*28])
x_test = tf.reshape(tf.cast(x_test, tf.float32), [-1, 28*28])

In [5]:
y_train = to_categorical(y_train, 10)

In [18]:
class DenseNN(tf.Module):
    def __init__(self, outputs, activate='relu'):
        super().__init__()
        self.outputs = outputs
        self.activate = activate
        self.fl_init = False
        
    def __call__(self, x):
        if not self.fl_init:
            self.w = tf.random.truncated_normal((x.shape[-1], self.outputs), stddev=0.1, name='w')  
            self.b = tf.zeros([self.outputs], dtype=tf.float32, name='b')
            
            self.w = tf.Variable(self.w)
            self.b = tf.Variable(self.b)
            
            self.fl_init = True
            
        y = x @ self.w + self.b
        
        if self.activate == 'relu':
            return tf.nn.relu(y)
        elif self.activate == 'softmax':
            return tf.nn.softmax(y)
        
        return y

In [19]:
layer_1 = DenseNN(128)
layer_2 = DenseNN(10, activate='softmax')

In [20]:
def model_predict(x):
    y = layer_1(x)
    y = layer_2(y)
    return y

In [21]:
cross_entropy = lambda y_true, y_pred: tf.reduce_mean(tf.losses.categorical_crossentropy(y_true, y_pred))

In [10]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

In [11]:
batch_size = 32
epochs = 10
total = x_train.shape[0]

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [25]:
for n in range(epochs):
    loss = 0
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            f_loss = cross_entropy(y_batch, model_predict(x_batch))
            
        loss += f_loss
        grads = tape.gradient(f_loss, [layer_1.trainable_variables, layer_2.trainable_variables])
        opt.apply_gradients(zip(grads[0], layer_1.trainable_variables))
        #opt.apply_gradients(zip(grads[1], layer_2.trainable_variables))
        
    print(loss.numpy())

402.81558
219.29317
162.4839
127.879745
103.4398
89.784744
70.23408
64.2272
51.867447
41.186726


In [26]:
y = model_predict(x_test)
y2 = tf.argmax(y, axis=1).numpy()
acc = len(y_test[y_test == y2]) / y_test.shape[0] * 100
print(acc)

96.77


In [27]:
class DenseNN(tf.Module):
    def __init__(self, outputs, activate='relu'):
        super().__init__()
        self.outputs = outputs
        self.activate = activate
        self.fl_init = False
        
    def __call__(self, x):
        if not self.fl_init:
            self.w = tf.random.truncated_normal((x.shape[-1], self.outputs), stddev=0.1, name='w')  
            self.b = tf.zeros([self.outputs], dtype=tf.float32, name='b')
            
            self.w = tf.Variable(self.w)
            self.b = tf.Variable(self.b, trainable=False) #необучаемый параметр. всегда равен 0
            
            self.fl_init = True
            
        y = x @ self.w + self.b
        
        if self.activate == 'relu':
            return tf.nn.relu(y)
        elif self.activate == 'softmax':
            return tf.nn.softmax(y)
        
        return y

In [13]:
for n in range(epochs):
    loss = 0
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            f_loss = cross_entropy(y_batch, model_predict(x_batch))
            
        loss += f_loss
        grads = tape.gradient(f_loss, [layer_1.trainable_variables, layer_2.trainable_variables])
        opt.apply_gradients(zip(grads[0], layer_1.trainable_variables))
        #opt.apply_gradients(zip(grads[1], layer_2.trainable_variables))
        
    print(loss.numpy())

NameError: name 'model_predict' is not defined

In [29]:
y = model_predict(x_test)
y2 = tf.argmax(y, axis=1).numpy()
acc = len(y_test[y_test == y2]) / y_test.shape[0] * 100
print(acc)

97.52


параметр b мало влияет на результат, поэтому асс практически не изменилось.

#### улучшенная нейронная сеть

In [6]:
class DenseNN(tf.Module):
    def __init__(self, outputs, activate='relu'):
        super().__init__()
        self.outputs = outputs
        self.activate = activate
        self.fl_init = False
        
    def __call__(self, x):
        if not self.fl_init:
            self.w = tf.random.truncated_normal((x.shape[-1], self.outputs), stddev=0.1, name='w')  
            self.b = tf.zeros([self.outputs], dtype=tf.float32, name='b')
            
            self.w = tf.Variable(self.w)
            self.b = tf.Variable(self.b)             
            self.fl_init = True
            
        y = x @ self.w + self.b
        
        if self.activate == 'relu':
            return tf.nn.relu(y)
        elif self.activate == 'softmax':
            return tf.nn.softmax(y)
        
        return y
    
class SequentialModule(tf.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = DenseNN(128)
        self.layer_2 = DenseNN(10, activate='softmax')
        
    def __call__(self, x):
        return self.layer_2(self.layer_1(x))

In [14]:
model = SequentialModule()

cross_entropy = lambda y_true, y_pred: tf.reduce_mean(tf.losses.categorical_crossentropy(y_true, y_pred))

#opt = tf.keras.optimizers.Adam(learning_rate=0.01)

batch_size = 32
epochs = 10
total = x_train.shape[0]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [15]:
for n in range(epochs):
    loss = 0
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            f_loss = cross_entropy(y_batch, model(x_batch))
            
        loss += f_loss
        grads = tape.gradient(f_loss, model.trainable_variables)
        opt.apply_gradients(zip(grads, model.trainable_variables))
    print(loss.numpy())

440.3526
299.5529
265.71024
244.24823
229.31699
219.79309
223.50606
196.91269
193.9259
189.78833


In [16]:
y = model(x_test)
y2 = tf.argmax(y, axis=1).numpy()
acc = len(y_test[y_test == y2]) / y_test.shape[0] * 100
print(acc)

96.38
